<a href="https://colab.research.google.com/github/nataliecar/elephant-counting/blob/main/STITCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

def resize_image(image, target_size):
    return cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

def image_stitching(image_folder, output_folder, target_size=(2446, 3669)):
    print(f"Input folder: {image_folder}")
    print(f"Output folder: {output_folder}")

    # Get a list of all image files in the folder
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    image_files.reverse()
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through each image in the folder
    for i, image_file in enumerate(image_files):
        # Read the current image
        current_image = cv2.imread(os.path.join(image_folder, image_file))

        print(f"Processing {image_file} - Type: {current_image.dtype}, Shape: {current_image.shape}")

        # Resize the image to the target size
        current_image = resize_image(current_image, target_size)

        # Initialize the stitched image for the current image
        stitched_image = current_image.copy()

        # Iterate through each other image in the folder
        for j, other_image_file in enumerate(image_files):
            if i != j:  # Skip comparing the image with itself
                # Read the other image
                other_image = cv2.imread(os.path.join(image_folder, other_image_file))

                # Resize the other image to the target size
                other_image = resize_image(other_image, target_size)

                # Check if the images have the same data type and number of columns
                if current_image.dtype != other_image.dtype or current_image.shape[1] != other_image.shape[1]:
                    print(f"Skipping {other_image_file} due to data type or dimension mismatch.")
                    print(f"Current Image Type: {current_image.dtype}, Shape: {current_image.shape}")
                    print(f"Other Image Type: {other_image.dtype}, Shape: {other_image.shape}")
                    continue

                # Find the keypoints and descriptors with ORB
                orb = cv2.ORB_create()
                kp1, des1 = orb.detectAndCompute(stitched_image, None)
                kp2, des2 = orb.detectAndCompute(other_image, None)

                # Use BFMatcher to find the best matches between descriptors
                bf = cv2.BFMatcher()
                matches = bf.knnMatch(des1, des2, k=2)

                # Apply ratio test
                good_matches = []
                for m, n in matches:
                    if m.distance < 0.6 * n.distance:
                        good_matches.append(m)

                # If enough good matches are found, stitch the images
                if len(good_matches) > 10:
                    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

                    M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

                    # Ensure stitched_image has the correct dimensions
                    stitched_image = cv2.warpPerspective(stitched_image, M, (other_image.shape[1], other_image.shape[0]))

                    stitched_image[0:other_image.shape[0], 0:other_image.shape[1]] = other_image

                    print(f"Stitched {image_file} with {other_image_file} - Matches: {len(good_matches)}")

        # Save the stitched image for the current image
        output_path = os.path.join(output_folder, f"stitched_{image_file}")
        cv2.imwrite(output_path, stitched_image.copy())  # Save a copy of the stitched image

    print("Image stitching completed.")

input_folder = '/content/drive/My Drive/AED/test_gsd/'
output_folder = '/content/drive/My Drive/Test_STITCH2/'
image_stitching(input_folder, output_folder)
